In [20]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [21]:
Dt = 6 #hours
n = 60 #days 
fp_delivery_interval = 7 #days
rm_delivery_interval = 14 #days
rm_num = 3
intm_num = 13
fp_num = 12

In [22]:
tasks_string  = "T111 T112 T121 T122 T123 T131 T132 T211 T212 T213 T214 T221 T222 T223 T224 T311 T312 T321 T322 T331 T332 T411 T412 T413 T414\
        T421 T422 T423 T424 T431 T432 T433 T434 T441 T442 T443 T444"
materials_string = "RM1 RM2 RM3 IN1A IN1B IN1CD IN2A IN2B IN2C IN2D IN3A1 IN3B1 IN3A2 IN3B2 IN3C IN3D A11 \
  B11 A21 B21 A12 B12 A22 B22 C1 D1 C2 D2"

tasks = tasks_string.split()
materials = materials_string.split()
units = ['U11', 'U12', 'U13', 'U21', 'U22', 'U31', 'U32', 'U33', 'U41', 'U42', 'U43', 'U44']
time_points = [i for i in range(int(n * (24 / Dt)) + 1)]
time_periods = [i for i in range(1, int(n * (24 / Dt)) + 1)]

In [23]:
capacities_string = "80.0 80.0 80.0 5.0 5.0 5.0 3.5 3.5 3.5 3.5 3.0 3.0 3.0 3.0 3.0 3.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0"
onhand_inventory_string = "35.0 50.0 40.0 3.0 3.0 3.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.5 1.5 1.5 3.0 3.0 3.0 3.0 2.0 2.0 2.0 2.0 3.0 3.0 3.0 3.0"
daily_production_rate_string = "1.50 1.50 1.00 1.00 1.00 1.00 1.00 1.50 1.50 1.50 1.50 1.50 1.50 1.50 1.50 1.00 1.00 1.50 1.50 1.50 1.50 1.00 1.00 1.00 1.00 0.50 0.50 0.50 0.50 0.75 0.75 0.75 0.75 1.00 1.00 1.00 1.00"
production_costs_string = "1.50 3.00 2.50 2.00 2.00 1.50 2.00 3.00 1.50 2.50 3.00 2.50 3.00 3.00 1.50 2.50 1.50 2.00 2.50 2.00 2.00 2.00 1.50 1.50 2.50 3.00 1.50 2.50 3.00 2.00 1.50 2.50 1.50 3.00 2.00 2.00 2.50"

In [24]:
capacities = [float(x) for x in capacities_string.split(' ')]
onhand_inventory = [float(x) for x in onhand_inventory_string.split(' ')]
production_costs = [float(x) for x in production_costs_string.split(' ')]
daily_production_rate = [float(x) for x in daily_production_rate_string.split(' ')]
production_rate = [rate * (Dt / 24) for rate in daily_production_rate]
storage_costs = [0 for i in range(16)] + [0.3, 0.5, 0.4, 0.8, 0.6, 0.6, 0.5, 0.8, 0.8, 1.2, 1, 1.5]

In [25]:
key_value_pairs_storage_cost = zip(materials, storage_costs)
key_value_pairs_production_cost = zip(tasks, production_costs)
key_value_pairs_capacity = zip(materials, capacities)
key_value_pairs_production_rate = zip(tasks, production_rate)
key_value_pairs_onhand_inventory = zip(materials, onhand_inventory)

material_storage_cost = dict(key_value_pairs_storage_cost)
task_production_cost = dict(key_value_pairs_production_cost)
capacity = dict(key_value_pairs_capacity)
rate = dict(key_value_pairs_production_rate)
onhand_inventory = dict(key_value_pairs_onhand_inventory)

In [26]:
rm_deliveries = {
  'RM1': [6.50, 7.00, 6.00],
  'RM2': [7.50, 10.00, 6.50],
  'RM3': [6.50, 10.50, 8.50]
}

fp_deliveries_string = "1.50 2.50 2.50 2.50 1.50 1.00 1.00 2.50 2.00 1.00 2.50 1.00\
                        1.00 2.00 2.50 1.00 1.00 2.00 2.50 1.50 1.50 1.50 1.50 1.00\
                        3.00 1.50 2.50 1.50 1.00 2.50 1.00 1.50 1.00 1.50 2.00 1.00\
                        1.00 1.50 1.50 1.50 3.00 3.00 1.50 1.50 2.50 1.50 3.00 1.00\
                        2.50 1.00 2.00 1.00 1.00 2.00 2.00 2.50 1.50 1.50 1.50 1.00\
                        3.00 2.00 1.00 1.50 1.50 1.50 3.00 1.00 1.50 1.00 2.00 1.00\
                        1.50 1.50 1.00 1.00 2.00 2.00 1.00 2.50 3.00 2.00 2.50 1.50\
                        2.00 2.50 2.00 1.00 2.50 2.00 1.00 3.00 1.50 2.50 2.50 3.00"

fp_deliveries = np.array([-float(x) for x in fp_deliveries_string.split()])
fp_deliveries = fp_deliveries.reshape(8, 12)
net_delivery = {}

for k in materials:
  for n in time_points[1:]:
    net_delivery[(k,n)] = 0

#net_delivery for final products
for i, k in enumerate(materials[16:]):
  for j, n in enumerate(range(int(fp_delivery_interval * (24 / Dt)), time_points[-1] + 1, int(fp_delivery_interval * (24 / Dt)))):
    net_delivery[(k,n)] = fp_deliveries[j][i]

#net_delivery for raw materials
for k in materials[:rm_num]:
  for i,n in enumerate(range(int(rm_delivery_interval * (24 / Dt) ), time_points[int(rm_delivery_interval * (24 / Dt) ) * 3 + 1], int(rm_delivery_interval * (24 / Dt) ))):
    net_delivery[(k,n)] = rm_deliveries[k][i]

In [27]:
for k in materials:
  for n in time_points[1:]:
    if (net_delivery[(k,n)]):
      print("net delivery(%s, %.2f): %.2f" % (k, n, net_delivery[(k,n)]))

net delivery(RM1, 56.00): 6.50
net delivery(RM1, 112.00): 7.00
net delivery(RM1, 168.00): 6.00
net delivery(RM2, 56.00): 7.50
net delivery(RM2, 112.00): 10.00
net delivery(RM2, 168.00): 6.50
net delivery(RM3, 56.00): 6.50
net delivery(RM3, 112.00): 10.50
net delivery(RM3, 168.00): 8.50
net delivery(A11, 28.00): -1.50
net delivery(A11, 56.00): -1.00
net delivery(A11, 84.00): -3.00
net delivery(A11, 112.00): -1.00
net delivery(A11, 140.00): -2.50
net delivery(A11, 168.00): -3.00
net delivery(A11, 196.00): -1.50
net delivery(A11, 224.00): -2.00
net delivery(B11, 28.00): -2.50
net delivery(B11, 56.00): -2.00
net delivery(B11, 84.00): -1.50
net delivery(B11, 112.00): -1.50
net delivery(B11, 140.00): -1.00
net delivery(B11, 168.00): -2.00
net delivery(B11, 196.00): -1.50
net delivery(B11, 224.00): -2.50
net delivery(A21, 28.00): -2.50
net delivery(A21, 56.00): -2.50
net delivery(A21, 84.00): -2.50
net delivery(A21, 112.00): -1.50
net delivery(A21, 140.00): -2.00
net delivery(A21, 168.00): -1

In [28]:
unit_can_process_tasks = {}
for unit in units:
    unit_list = []
    for task in tasks:
        if(unit[1:] == task[1:3]):
            unit_list.append(task)
        elif(len(unit_list)):
            break
    unit_can_process_tasks[unit] = unit_list

In [29]:
materials_produced_by_tasks = {
  'RM1': [],
  'RM2': [],
  'RM3': [],
  'IN1A': ['T111', 'T121'],
  'IN1B': ['T112', 'T122', 'T131'],
  'IN1CD': ['T123', 'T132'],
  'IN2A': ['T211', 'T221'],
  'IN2B': ['T212', 'T222'],
  'IN2C': ['T213', 'T223'],
  'IN2D': ['T214', 'T224'],
  'IN3A1': ['T311'],
  'IN3B1': ['T312'],
  'IN3A2': ['T321'],
  'IN3B2': ['T322'],
  'IN3C': ['T331'],
  'IN3D': ['T332'],
  'A11': ['T411'],
  'B11': ['T412'],
  'A21': ['T413'],
  'B21': ['T414'],
  'A12': ['T421', 'T431'],
  'B12': ['T422', 'T432'],
  'A22': ['T422', 'T433'],
  'B22': ['T424', 'T434'],
  'C1': ['T441'],
  'D1': ['T442'],
  'C2': ['T443'],
  'D2': ['T444']
}

materials_consumed_by_tasks =  {
  'RM1': ['T111', 'T121'],
  'RM2': ['T112', 'T122', 'T131'],
  'RM3': ['T123', 'T132'],
  'IN1A': ['T211', 'T221'],
  'IN1B': ['T212', 'T222'],
  'IN1CD': ['T213', 'T214', 'T223', 'T224'],
  'IN2A': ['T311', 'T321'],
  'IN2B': ['T312', 'T322'],
  'IN2C': ['T331'],
  'IN2D': ['T332'],
  'IN3A1': ['T411', 'T421', 'T431'],
  'IN3B1': ['T412', 'T422', 'T432'],
  'IN3A2': ['T413', 'T423', 'T433'],
  'IN3B2': ['T414', 'T424', 'T434'],
  'IN3C': ['T441', 'T442'],
  'IN3D': ['T443', 'T444'],
  'A11': [],
  'B11': [],
  'A21': [],
  'B21': [],
  'A12': [],
  'B12': [],
  'A22': [],
  'B22': [],
  'C1': [],
  'D1': [],
  'C2': [],
  'D2': []
  } 
  


In [30]:
m = gp.Model("m2-inst1")

In [31]:
#Define model variables
w = m.addVars(tasks, time_periods, vtype=GRB.BINARY, name="w")
w_s = m.addVars(units, time_periods, vtype=GRB.BINARY, name="w_s")

s = m.addVars(materials, time_points[1:], vtype=GRB.CONTINUOUS, lb=0, name="s")

In [32]:
#Define objective function
obj = sum( material_storage_cost[k] * Dt / 24 * s[k,n] for k in materials for n in time_points[1:]) + \
      sum(task_production_cost[i] * rate[i] * w[i,n] for n in time_periods for i in tasks)

m.setObjective(obj, GRB.MINIMIZE)

In [33]:
#Define constraints
capacity_constraint = m.addConstrs(s[k,n] <= capacity[k] for k in materials for n in time_points[1:])

assignment = m.addConstrs(sum(w[i,n] for i in unit_can_process_tasks[j]) + w_s[j,n] == 1 \
  for j in units for n in time_periods)

initial_balance = m.addConstrs(s[k,1] == onhand_inventory[k] + net_delivery[k,1] \
  + sum(rate[i] * w[i,1] for i in materials_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,1] for i in materials_consumed_by_tasks[k]) \
                             for k in materials )
                             
balance = m.addConstrs(s[k,n] == s[k,n-1] + net_delivery[k,n] \
  + sum(rate[i] * w[i,n] for i in materials_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,n] for i in materials_consumed_by_tasks[k]) \
  for k in materials for n in time_points[2:] )
            

In [34]:
m.write('m1-inst2.lp' )

In [35]:
lp_m = m.relax()

In [36]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16320 rows, 18480 columns and 49652 nonzeros
Model fingerprint: 0x438f637a
Variable types: 6720 continuous, 11760 integer (11760 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [7e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 10154 rows and 6329 columns
Presolve time: 0.32s
Presolved: 6166 rows, 12151 columns, 33169 nonzeros
Variable types: 0 continuous, 12151 integer (8880 binary)

Root relaxation: objective 1.493062e+03, 5272 iterations, 0.37 seconds (0.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1493.06250    0  122  

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\amoum\AppData\Roaming\Python\Python310\site-packages\ipykernel\iostream.py", line 502, in write
    def write(self, string: str) -> int:
KeyboardInterrupt: 


  5557  5504 1496.02684  269  164          - 1496.02684      -   6.9   45s
  5572  5514 1496.03224  169  503          - 1496.03224      -   6.9   50s


In [ ]:
lp_m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24480 rows, 27720 columns and 74492 nonzeros
Model fingerprint: 0x95ef6d8e
Coefficient statistics:
  Matrix range     [8e-02, 1e+00]
  Objective range  [5e-02, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 15224 rows and 9467 columns
Presolve time: 0.10s
Presolved: 9256 rows, 18253 columns, 49801 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 3.942e+04
 Factor NZ  : 2.252e+05 (roughly 13 MB of memory)
 Factor Ops : 6.429e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time

In [ ]:
for v in lp_m.getVars():
    print(f"{v.VarName} = {v.X}")

w[T111,1] = 1.0
w[T111,2] = 1.0
w[T111,3] = 0.0
w[T111,4] = 1.0
w[T111,5] = 0.0
w[T111,6] = 0.0
w[T111,7] = 1.0
w[T111,8] = 0.0
w[T111,9] = 1.0
w[T111,10] = 0.0
w[T111,11] = 1.0
w[T111,12] = 0.0
w[T111,13] = 0.0
w[T111,14] = 1.0
w[T111,15] = 0.0
w[T111,16] = 1.0
w[T111,17] = 0.0
w[T111,18] = 0.0
w[T111,19] = 0.0
w[T111,20] = 1.0
w[T111,21] = 0.0
w[T111,22] = 1.0
w[T111,23] = 0.0
w[T111,24] = 0.0
w[T111,25] = 0.0
w[T111,26] = 1.0
w[T111,27] = 0.0
w[T111,28] = 1.0
w[T111,29] = 0.0
w[T111,30] = 0.0
w[T111,31] = 0.0
w[T111,32] = 1.0
w[T111,33] = 0.0
w[T111,34] = 0.0
w[T111,35] = 0.0
w[T111,36] = 0.0
w[T111,37] = 0.0
w[T111,38] = 1.0
w[T111,39] = 0.0
w[T111,40] = 1.0
w[T111,41] = 0.0
w[T111,42] = 0.0
w[T111,43] = 1.0
w[T111,44] = 0.0
w[T111,45] = 0.0
w[T111,46] = 0.0
w[T111,47] = 1.0
w[T111,48] = 0.0
w[T111,49] = 1.0
w[T111,50] = 0.0
w[T111,51] = 0.0
w[T111,52] = 0.0
w[T111,53] = 0.0
w[T111,54] = 0.0
w[T111,55] = 0.0
w[T111,56] = 1.0
w[T111,57] = 1.0
w[T111,58] = 0.0
w[T111,59] = 0.0
w[T111